# Price impact estimation

In this notebook, we show how you can estimate the price impact, also known as slippage, of the simulated historical decentralised exchange trades.

The slippage is especially important when trading illiquid assets. If your trade profit is 2% but the sell slippage is also 2% you have essentially not made any profit.

## About price impact and slippage on AMMs

The slippage is a function of

* Available liquidity
* Liquidity provider fees
* Protocol fees

Slippage, also known as price impact, tells how much less your market taker order gets filled because there is not available liquidity.
For example, if you are trying to buy 5000 USD worth of BNB token, but there isn't available liquidity
you end up with 4980 USD worth of token at the end of the trade. The missing fill is called slippage.
It can be expressed as USD value or as percent of the trade amount.
Illiquid pairs have more slippage than liquid pairs.

In Uniswap v2 style decentralised exchanges any fees are included in the slippage calculation.

For more information read more about [XY=K liquidity model and automated market makers](https://tradingstrategy.ai/docs/glossary.html#term-XY-liquidity-model).

## Getting started

First let's import libraries and initialise our Trading Strategy dataset client.

In [26]:
try:
    import tradingstrategy
except ImportError:
    !pip install tradingstrategy
    import site
    site.main()

from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


## Fetching datasets

Here we fetch the liquidity data for

* [SUSHI-ETH trading pair](https://tradingstrategy.ai/trading-view/ethereum/sushiswap/sushi-eth)
* [Trading on Sushiswap](https://tradingstrategy.ai/trading-view/ethereum/sushiswap)
* [On Ethereum mainnet](https://tradingstrategy.ai/trading-view/ethereum)

To make this notebook run fast, we are operating on weekly candles (7 days) to keep the
download size of datasets smaller.

In [ ]:

from tradingstrategy.chain import ChainId
from tradingstrategy.liquidity import GroupedLiquidityUniverse
from tradingstrategy.pair import PandasPairUniverse
from tradingstrategy.timebucket import TimeBucket

exchange_universe = client.fetch_exchange_universe()
raw_pairs = client.fetch_pair_universe().to_pandas()
raw_liquidity_samples = client.fetch_all_liquidity_samples(TimeBucket.d7).to_pandas()

pair_universe = PandasPairUniverse(raw_pairs)
liquidity_universe = GroupedLiquidityUniverse(raw_liquidity_samples)

# Do some test calculations for a single pair
sushi_swap = exchange_universe.get_by_chain_and_name(ChainId.ethereum, "sushiswap")
sushi_eth = pair_universe.get_one_pair_from_pandas_universe(sushi_swap.exchange_id, "SUSHI", "WETH")

## Querying historical price impact

Here we get the historical price impact what would have been for buying SUSHI for 6000 USD worth of ETH on 1st of June 2021.

In [28]:
import pandas as pd
from IPython.display import display
from tradingstrategy.priceimpact import HistoricalXYPriceImpactCalculator

# SUSHI-WETH started trading around 2020-09-01
trading_date = pd.Timestamp("2021-06-01")
trade_size = 6000.0  # USD

price_impact_calculator = HistoricalXYPriceImpactCalculator(liquidity_universe)
impact = price_impact_calculator.calculate_price_impact(trading_date, sushi_eth.pair_id, trade_size, max_distance=pd.Timedelta(days=7))

display(impact)

## Possible errors

Because the lack of liquidity data, the price impact calculation may fail. See